# 34. 元类（Metaclasses）

元类控制“类的创建过程”。多数业务场景不需要元类；优先考虑装饰器、工厂函数、__init_subclass__。本节目标是能读懂并知道何时不用。

> 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行。


## 前置知识

- 第 27 节：类高级主题（super/MRO/钩子思想）


## 知识点地图

- 1. type：默认元类
- 2. 元类的作用：在创建类时改造类
- 3. 注册型元类示例：自动登记子类
- 4. 更简单替代：__init_subclass__
- 5. __prepare__（了解）：控制类体命名空间


## 自检清单（学完打勾）

- [ ] 理解类也是对象，默认元类是 type
- [ ] 了解 metaclass=... 如何介入类创建
- [ ] 能读懂一个简单的注册型元类
- [ ] 了解 __init_subclass__ 作为更简单替代方案


## 知识点 1：type：默认元类

class 语句最终会调用元类创建类对象；默认元类是 type。


In [ ]:
MyClass = type('MyClass', (), {'x': 1})
obj = MyClass()
print(obj.x)
print(isinstance(MyClass, type))


## 知识点 2：元类的作用：在创建类时改造类

元类可用于：自动注册、自动加字段/方法、校验类定义等（多见于框架/ORM）。


## 知识点 3：注册型元类示例：自动登记子类

在 __new__ 里把新创建的类放入 registry。


In [ ]:
class RegistryMeta(type):
    registry = {}

    def __new__(mcls, name, bases, namespace):
        cls = super().__new__(mcls, name, bases, namespace)
        if name != 'PluginBase':
            mcls.registry[name] = cls
        return cls

class PluginBase(metaclass=RegistryMeta):
    pass

class PluginA(PluginBase):
    pass

class PluginB(PluginBase):
    pass

print(RegistryMeta.registry)


## 知识点 4：更简单替代：__init_subclass__

__init_subclass__ 在子类创建时自动调用，常可替代元类完成注册。


In [ ]:
class Base:
    registry = {}

    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        Base.registry[cls.__name__] = cls

class A(Base):
    pass

class B(Base):
    pass

print(Base.registry)


## 知识点 5：__prepare__（了解）：控制类体命名空间

更高级用法：控制 class 体内部 namespace（有序字典等）。一般业务不需要。


## 常见坑

- 元类复杂度高，维护成本大；能不用就不用
- 优先考虑 __init_subclass__/装饰器/工厂函数等简单方案


## 综合小案例：按插件 name 属性注册（元类版）

在元类里读取类属性 name 作为注册 key；未设置则用类名。


In [ ]:
class RegistryMeta(type):
    registry = {}

    def __new__(mcls, name, bases, namespace):
        cls = super().__new__(mcls, name, bases, namespace)
        if name != 'PluginBase':
            key = getattr(cls, 'name', cls.__name__)
            mcls.registry[key] = cls
        return cls

class PluginBase(metaclass=RegistryMeta):
    pass

class JsonPlugin(PluginBase):
    name = 'json'

class XmlPlugin(PluginBase):
    name = 'xml'

print(RegistryMeta.registry)


## 自测题（不写代码也能回答）

- 为什么说类也是对象？
- 元类最常见的用途是什么？
- __init_subclass__ 为什么能替代很多元类场景？


## 练习题（建议写代码）

- 用 __init_subclass__ 实现同样的按 name 注册，并对比实现复杂度。
- 思考：装饰器能否实现注册？写一个装饰器版 register(name)。
